<font size="5">**Biotechology. Practice 2.**</font>

**Цель:**
- Познакомиться с основными библиотеками для работы с материалами и биологическими последовательностями

<font size="4">**Chapter 1.  Материалы**</font>

In [2]:
import os
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

MP_API_KEY = os.getenv('MP_API_KEY')

In [ ]:
from mp_api.client import MPRester
from pymatgen.core import Structure

with MPRester(MP_API_KEY) as mpr:
    doc = mpr.materials.summary.search(
        formula="MnFe2O4",
        fields=["material_id", "formula_pretty", "structure"]
    )[0]
    mid = doc.material_id
    s = doc.structure

pprint(f"Material ID: {mid}")
pprint(f"Formula: {s.composition.reduced_formula}")


Retrieving SummaryDoc documents: 100%|██████████| 6/6 [00:00<00:00, 107088.61it/s]

'Material ID: mp-697842'
'Formula: Mn(FeO2)2'


In [ ]:
# Состав
comp = s.composition
pprint(f"Состав (элементы и количества): {comp.get_el_amt_dict()}")
pprint(f"Молярная масса, г/моль: {comp.weight}")

# Параметры решётки и плотность (местами ооочень незнакомые)
lat = s.lattice
print(f"a,b,c (Å): {lat.a:.4f}, {lat.b:.4f}, {lat.c:.4f}")
print(f"α,β,γ (°): {lat.alpha:.2f}, {lat.beta:.2f}, {lat.gamma:.2f}")
print("Объём ячейки (A^3):", round(lat.volume, 4))
print("Плотность (г/см^3):", round(s.density, 4))




"Состав (элементы и количества): {'Mn': 10.0, 'Fe': 20.0, 'O': 40.0}"
'Молярная масса, г/моль: 2306.2564500000003 amu'
a,b,c (Å): 6.1505, 6.1007, 30.3960
α,β,γ (°): 60.28, 60.00, 59.84
Объём ячейки (A^3): 807.1544
Плотность (г/см^3): 4.7446


<font size="4">**Chapter 2.  Последовательности ДНК/РНК**</font>

In [17]:
from Bio.Seq import  Seq
from Bio.SeqUtils import gc_fraction

seqs = {
    "seq1": Seq("ATGCATGCCGTAGTC"),
    "seq2": Seq("TTGAAAACTGTACTC"),
}

for name, seq in seqs.items():
    gc = gc_fraction(seq) * 100 
    rc = seq.reverse_complement()
    rna = seq.transcribe() # T -> U
    print(f"{name}: len={len(s)}, GC%={gc:.2f}")
    print(" DNA:", str(s))
    print(" rev-compl:", str(rc))
    print(" RNA:", str(rna))
    print("-"*40)

seq1: len=70, GC%=53.33
 DNA: Full Formula (Mn10 Fe20 O40)
Reduced Formula: Mn(FeO2)2
abc   :   6.150494   6.100717  30.396005
angles:  60.279723  59.998079  59.837294
pbc   :       True       True       True
Sites (70)
  #  SP           a         b         c    magmom
---  ----  --------  --------  --------  --------
  0  Mn    0.125585  0.12847   0.02481      4.639
  1  Mn    0.002381  0.500143  0.099514     3.989
  2  Mn    0.128669  0.127284  0.224312     4.634
  3  Mn    0.128352  0.122436  0.425415     4.631
  4  Mn    0.000682  0.497553  0.50056      4.116
  5  Mn    0.12062   0.124349  0.627158     4.63
  6  Mn    0.002588  0.501439  0.299683     4.492
  7  Mn    0.122237  0.127152  0.825253     4.642
  8  Mn    0.998346  0.502359  0.700177     4.095
  9  Mn    0.878815  0.874438  0.9738       4.649
 10  Fe    0.502299  0.001466  0.099341     4.369
 11  Fe    0.500029  0.503541  0.999434     4.38
 12  Fe    0.500882  0.499626  0.099371     4.367
 13  Fe    0.497122  0.005267  0

<font size="4">**Chapter 3.  Белки**</font>

In [19]:
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import requests
from io import StringIO
from io import StringIO
from collections import Counter
from itertools import product


In [21]:
uniprot_id = "P04746"  # альфа-амилаза 
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
resp = requests.get(url)
resp.raise_for_status()

seq_record = SeqIO.read(StringIO(resp.text), "fasta")
seq = str(seq_record.seq)

print("UniProt ID:", uniprot_id)
print("Длина последовательности:", len(seq))


UniProt ID: P04746
Длина последовательности: 511


In [23]:
pa = ProteinAnalysis(seq)

aa_counts = pa.count_amino_acids()
aa_perc = pa.get_amino_acids_percent()

print("Состав аминокислот:", "\n")
for aa in sorted(aa_counts.keys()):
    print(f" {aa}: {aa_counts[aa]} ({aa_perc[aa]*100:.2f}%)")

# частоты дипептидов, проще в доке что то не нашел(((
aas20 = list("ACDEFGHIKLMNPQRSTVWY")
total_dp = len(seq) - 1
dp_counts = Counter(
    seq[i:i+2]
    for i in range(len(seq) - 1)
    if seq[i] in aas20 and seq[i+1] in aas20
)

print("Топ-10 наиболее частых дипептидов:", "\n")
top10 = dp_counts.most_common(10)
for dp, cnt in top10:
    print(f" {dp}: {cnt} ({cnt/total_dp:.4f})")

Состав аминокислот: 

 A: 28 (5.48%)
 C: 12 (2.35%)
 D: 34 (6.65%)
 E: 19 (3.72%)
 F: 30 (5.87%)
 G: 51 (9.98%)
 H: 11 (2.15%)
 I: 29 (5.68%)
 K: 23 (4.50%)
 L: 28 (5.48%)
 M: 10 (1.96%)
 N: 42 (8.22%)
 P: 23 (4.50%)
 Q: 14 (2.74%)
 R: 28 (5.48%)
 S: 32 (6.26%)
 T: 22 (4.31%)
 V: 36 (7.05%)
 W: 18 (3.52%)
 Y: 21 (4.11%)
Топ-10 наиболее частых дипептидов: 

 GF: 7 (0.0137)
 GN: 7 (0.0137)
 VI: 6 (0.0118)
 ND: 6 (0.0118)
 NG: 6 (0.0118)
 AG: 5 (0.0098)
 GS: 5 (0.0098)
 VD: 4 (0.0078)
 DI: 4 (0.0078)
 GV: 4 (0.0078)
